---
## Imports
---

In [4]:
import pandas as pd
import os
import json
from web3 import Web3
from pathlib import Path
from dotenv import load_dotenv
import streamlit as st
import time
from datetime import datetime

load_dotenv()

True

---
## Server Funcitonality
---

In [5]:
# Pull Log
# Evaluate traders
	# Trading period
	# Trading metrics
	# Send Eval Report (open source/show functions)
	
# Distribute Subscriptions


---
## Client Functionality
---

### Trader Functionality


In [6]:
from Libraries.web3_contract import load_web3
contract,address,w3 = load_web3(".\Contracts\ABI\Contract_abi.json")
address

'0x12e8e73231Dd04547C1c50e6f0B0E9a6cBfA4d08'

In [7]:
# For interacting with the market (Opening/Closing Trades) utilize this method
# Easy to keep things consistent and trade out trading platforms
from datetime import datetime
from Libraries.TradingPlatforms import trade_platforms, init_TradingPlatform
test_platform = init_TradingPlatform(trade_platforms['alpaca'],contract)
test_platform.__hello__()

'This is a Alpaca trading platform.'

In [8]:
TraderAddress = address
TraderID= 1
Open= 1
Symbol= "BTC"
Size= 1
EntryPrice= test_platform.platform.trade_api #TODO get price
EntryTime= datetime.now()
ExpirationTimeStamp= "NOT USED"
Strike= "NOT USED"
IsCall= "NOT USED"

test_platform.openTrade(TraderAddress,TraderID,Open,Symbol,Size,EntryPrice,EntryTime,ExpirationTimeStamp,Strike,IsCall)

AttributeError: 'str' object has no attribute 'trade_api'

### Viewer Functionality


In [ ]:
# Subscribe

---
## Client Interface
---


### Trader Interface


In [1]:
import alpaca_trade_api as tradeapi

API_KEY = os.getenv("APCA_API_KEY_ID")
API_SECRET = os.getenv("APCA_API_SECRET_KEY")
BASE_URL = os.getenv("APCA_API_BASE_URL")
alpaca_trader = tradeapi.REST(API_KEY, API_SECRET, BASE_URL)

In [2]:
alpaca_trader.get_bars("AAPL",'1Min',limit=1).df['close'][0]

160.44

In [14]:
alpaca_trader.list_orders()[0].status in ["accepted","accepted_for_bidding","calculated","done_for_day","filled","new","partially_filled"]

True

In [3]:
########## SideBar ##########
# list(trade_platforms.values())
# Select Trade Type Dropdown
# Start Trading Button

########## Tabs ##########
# Enter Trade
# Close Trade (from radio or dropdown)

EntryPrice=2.50
EntryTime=int(time.mktime(datetime.now().timetuple()))
theList = [EntryPrice,EntryTime]


NameError: name 'time' is not defined

In [ ]:
def list_to_string(inputList):
    outString = ""
    for item in inputList:
        if type(item) == datetime:
            outString += int(time.mktime(item.timetuple()))
        else:
            outString += str(item)
        outString
        if (item != inputList[len(inputList)-1]):
            outString += ", "
    return outString
theString = list_to_string(theList)
theString

'2.5, 1659204818'

In [ ]:
def string_to_list_end_dateTime(inputString):
    splitList = inputString.split(", ")
    outList = []
    for item in splitList:
        if (item != splitList[len(splitList)-1]):
            outList.append(float(item))
        else:
            outList.append(pd.to_datetime(item, unit='s'))
    return outList
otherEnd = string_to_list_end_dateTime(theString)
otherEnd

[2.5, Timestamp('2022-07-30 18:13:38')]

In [ ]:
def input_date_string(date):
    return int(time.mktime(pd.to_datetime(date).timetuple()))
input_date_string(datetime.now().strftime("%m/%d/%Y, %H:%M:%S"))

1659399117

### Viewer Interface

In [ ]:
# Enter Subscription
# View Subscriptions
	# Filter from logger and grader
